In [ ]:
%matplotlib inline
import flybrainlab.circuit as circuit
import vistrans.config as config
import matplotlib.pyplot as plt

In [ ]:
client = fbl.get_client()

Display all photoreceptors

In [ ]:
res = client.executeNLPquery('show all')

Retrieve the circuit model linked to the photoreceptors

In [ ]:
c = circuit.ExecutableCircuit(client, res, 'retina', '1.0')

Disable all but R1 photoreceptors

In [ ]:
c.disable_neurons(['R{}-{}'.format(k, i) for i in range(721) for k in range(2,9)])

Get photoreceptor parameters

In [ ]:
photoreceptors = {rid: v for rid, v in c.graph.nodes(data=True) if v['class'] == 'PhotoreceptorModel' and v['uname'] in c.config['active']['neuron']}

Specify input processor and what data to record (output processor)

In [ ]:
input_processors = {'RET(R)': [{'class': 'RetinaInputIndividual',
                            'name': 'RET',
                            'module': 'vistrans.InputProcessors.RetinaInputIndividual', # this is where PB_InputProcessor should be imported from
                            'input_config': config.Input().to_dict(),
                            'photoreceptors': photoreceptors,
                            'dt': 1e-4,
                            'radius': 1.0,
                            'input_file': 'retina_input.h5',
                            'input_interval': 10}]
                   }
output_processors = {'RET(R)': [{'class': 'Record',
                             'uid_dict': {'V': {'uids': None},
                                          },
                             'sample_interval': 10}
                           ]
                    }

Execute circuit

In [ ]:
c.execute(input_processors = input_processors,
          output_processors = output_processors,
          steps = 10000, dt = 1e-4)

Results are automatically retrieved after simulation is complete. The following method reorganize the naming of inputs/outputs so they are easy to call

In [ ]:
result = c.get_result()

Plot input and output of one of the photoreceptor (R1-0)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(result['input']['R1-0']['photon']['data'])
ax[1].plot(result['output']['R1-0']['V']['data'])